In [1]:
from typing import Optional, Sequence, Tuple
import requests
import json

SYSTEM = "You are a helpful assistant."


def chat_complete(
    *,
    system: str = SYSTEM,
    history: Sequence[Tuple[str, str]] = (),
    question: str = "",
    api_key_path: str = "api_key.txt",
    proxy: Optional[str] = None,
):
    messages = [{"role": "system", "content": system}]
    for user, assistant in history:
        messages.extend(
            (
                {"role": "user", "content": user},
                {"role": "assistant", "content": assistant},
            )
        )
    messages.append({"role": "user", "content": question})

    url = "https://api.openai.com/v1/chat/completions"
    payload = {"model": "gpt-3.5-turbo", "messages": messages, "stream": True}

    with open(api_key_path) as f:
        api_key = f.read().strip()

    headers = {"Authorization": f"Bearer {api_key}"}

    response = requests.post(
        url,
        json=payload,
        headers=headers,
        proxies={"https": proxy} if proxy else None,
        stream=True,
    )

    for line in response.iter_lines(decode_unicode=False):
        line = line.decode("utf-8")
        if line.startswith("data:"):
            if line == "data: [DONE]":
                return
            if data := line[5:]:
                yield json.loads(data)["choices"][0]["delta"].get("content", "")


In [2]:
topics = [
    "Politics",
    "Immigration",
    "Climate Change",
    "Environment",
    "Technology",
    "Social Media",
    "Privacy",
    "Art",
    "Culture",
    "Sports",
    "Health",
    "Education",
    "Media",
    "Science",
    "Religion",
    "Gender Equality",
    "Race Relations",
    "War and Military",
    "International Relations",
    "Economy and Finance",
    "Food and Diet",
    "Travel and Adventure",
    "Animal Rights and Protection",
    "Mental Health",
    "Physical Health",
    "Crime and Justice",
    "Marriage and Family",
    "Fashion and Beauty",
    "Entertainment",
    "Personal Wealth",
    "Charity",
    "Human Rights",
    "Social Justice",
    "Science Fiction and Fantasy",
    "Music",
    "Pop Culture",
    "Infrastructure",
    "Urbanization",
    "Natural Disasters",
    "Emergency Relief",
    "Professional Sports",
    "Energy Efficiency",
    "Sustainability",
    "Career Development",
    "Talent Acquisition",
    "Computer Science",
    "Programming",
    "Mathematics",
    "Statistics",
    "Fine Art",
    "Collectibles",
    "Film and Television",
    "Military History and Technology",
    "Architecture and Design",
    "Audio and Sound Technology",
    "Shopping and Consumer Behavior",
    "Technology Entrepreneurship",
    "Startups",
    "Personal Development",
    "Government and Governance",
    "Parenting and Family",
    "Law and Regulation",
    "Artificial Intelligence",
    "Machine Learning",
    "Interpersonal Relationships and Communication",
]
print(len(topics))


65


In [3]:
import json


def gen_attributes(topic: str, template: str):
    question = (
        """
    一组数据的三个维度分别是类别、时间范围、数值范围，请给出一些这样的数据属性。类别属性里的choice应该包含5到10个。
    例如：[[TEMPLATE]]
    请避免与我给出的句子重复, 你应该替换为 [[TOPIC]] 的知识。
    三个数据维度在意义上应该有关联，只输出一个 json，不要输出额外的内容。
    """.strip()
        .replace("[[TOPIC]]", topic)
        .replace("[[TEMPLATE]]", template)
    )

    try:
        s = ''.join(
            chat_complete(
                system="You are a helpful assistant that process json well.",
                question=question,
                proxy="http://127.0.0.1:7890",
            )
        )

        start, end = s.find("{"), len(s) - s[::-1].find("}")

        attributes = json.loads(s[start:end])
        attributes = next(iter(attributes.values()))        

        return attributes
    except Exception:
        return None


In [5]:
from typing import Union
from pydantic import BaseModel


class Categorical(BaseModel):
    type: str
    name: str
    choice: Sequence[str]


class Temporal(BaseModel):
    type: str
    span: Union[Sequence[str], Sequence[int]]


class Quantitative(BaseModel):
    type: str
    name: str
    range: Union[Sequence[str], Sequence[int]]


In [7]:
import contextlib
import random
import tqdm
import pandas as pd

N = 500

df = pd.DataFrame(columns=["topic", "c", "t", "q"])

template_ctq = """{"attribute": [{"type": "categorical","name": "country","choice": ["China", "Japan", "USA", "Germany", "France", "Iran"]}, {"type": "temporal","span": [2010, 2023]},{"type": "quantitative","name": "GDP","range": [10, 2000]}]}"""

success = 0
error = 0

with tqdm.tqdm(total=N) as pbar:
    for _ in range(N):
        topic = random.choice(topics)

        if attributes := gen_attributes(topic, template_ctq):
            c, t, q = attributes
            with contextlib.suppress(Exception):
                c = Categorical(**c).dict()
                t = Temporal(**t).dict()
                q = Quantitative(**q).dict()
                df = pd.concat(
                    [df, pd.DataFrame({"topic": [topic], "c": [c], "t": [t], "q": [q]})],
                    ignore_index=True,
                )
                success += 1
                pbar.set_description(f"success: {success}, error: {error}")
                pbar.update(1)
                df.to_csv("ctq_attribute.csv", index=False)
                continue
        error += 1
        pbar.set_description(f"success: {success}, error: {error}")
        pbar.update(1)



success: 168, error: 4:  34%|███▍      | 172/500 [35:16<1:07:16, 12.31s/it]


KeyboardInterrupt: 

In [8]:
import pandas as pd

pd.read_csv("ctq_attribute.csv").head(10)

,topic,c,t,q
0,Technology,"{'type': 'categorical', 'name': 'industry', 'c...","{'type': 'temporal', 'span': ['2015', '2025']}","{'type': 'quantitative', 'name': 'Revenue', 'r..."
1,Startups,"{'type': 'categorical', 'name': 'industry', 'c...","{'type': 'temporal', 'span': ['2015', '2021']}","{'type': 'quantitative', 'name': 'Funding', 'r..."
2,Mathematics,"{'type': 'categorical', 'name': 'mathematical ...","{'type': 'temporal', 'span': ['1990', '2020']}","{'type': 'quantitative', 'name': 'number of pu..."
3,Marriage and Family,"{'type': 'categorical', 'name': 'relationship_...","{'type': 'temporal', 'span': ['1980', '2023']}","{'type': 'quantitative', 'name': 'number_of_ch..."
4,Physical Health,"{'type': 'categorical', 'name': 'activity', 'c...","{'type': 'temporal', 'span': ['January 1, 2020...","{'type': 'quantitative', 'name': 'Calories Bur..."
5,Sustainability,"{'type': 'categorical', 'name': 'industry', 'c...","{'type': 'temporal', 'span': ['2000', '2025']}","{'type': 'quantitative', 'name': 'Carbon Emiss..."
6,Travel and Adventure,"{'type': 'categorical', 'name': 'destination',...","{'type': 'temporal', 'span': ['2022 Spring', '...","{'type': 'quantitative', 'name': 'budget', 'ra..."
7,Parenting and Family,"{'type': 'categorical', 'name': 'Parenting Sty...","{'type': 'temporal', 'span': ['1990', '2021']}","{'type': 'quantitative', 'name': 'Number of Ch..."
8,Education,"{'type': 'categorical', 'name': 'degree', 'cho...","{'type': 'temporal', 'span': ['1990', '2022']}","{'type': 'quantitative', 'name': 'Salary', 'ra..."
9,War and Military,"{'type': 'categorical', 'name': 'country', 'ch...","{'type': 'temporal', 'span': ['1939', '1945']}","{'type': 'quantitative', 'name': 'casualties',..."


In [28]:
templates = []


def template(t: str):
    def wrapper(f):
        templates.append((t, f))
        return f

    return wrapper


# @template("What is the {quantity} of {category_1}?")
# def cq_identify(category, category_1, quantity, **_):
#     return {
#         "identify": quantity,
#         "filter": [{"attr": category, "op": "=", "value": category_1}],
#     }


# @template("Which {category} has the highest {quantity}?")
# def cq_max(category, quantity, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {
#                 "attr": quantity,
#                 "op": "=",
#                 "value": {"operation": "max", "attribute": quantity},
#             }
#         ],
#     }


# @template("Which {category} has the lowest {quantity}?")
# def cq_min(category, quantity, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {
#                 "attr": quantity,
#                 "op": "=",
#                 "value": {"operation": "min", "attribute": quantity},
#             }
#         ],
#     }


# @template("What is the difference of {quantity} between {category_1} and {category_2}?")
# def cq_diff(category, category_1, category_2, quantity, **_):
#     return {
#         "compare": quantity,
#         "list": [
#             {
#                 "identify": quantity,
#                 "filter": [{"attr": category, "op": "=", "value": category_1}],
#             },
#             {
#                 "identify": quantity,
#                 "filter": [{"attr": category, "op": "=", "value": category_2}],
#             },
#         ],
#     }


# @template("Which {category} has the {quantity} that is above average?")
# def cq_above_avg(category, quantity, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {
#                 "attr": quantity,
#                 "op": ">",
#                 "value": {"operation": "avg", "attribute": quantity},
#             }
#         ],
#     }


# @template("Which {category} has the {quantity} that is below average?")
# def cq_below_avg(category, quantity, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {
#                 "attr": quantity,
#                 "op": "<",
#                 "value": {"operation": "avg", "attribute": quantity},
#             }
#         ],
#     }


# @template("what is the sum of {quantity} of {category_1} and {category_2}?")
# def cq_sum(category, category_1, category_2, quantity, **_):
#     return {
#         "sum": quantity,
#         "list": [
#             {
#                 "identify": quantity,
#                 "filter": [{"attr": category, "op": "=", "value": category_1}],
#             },
#             {
#                 "identify": quantity,
#                 "filter": [{"attr": category, "op": "=", "value": category_2}],
#             },
#         ],
#     }


# @template("What is the {quantity} of {category_1} in {time_1}?")
# def ctq_identify(category, category_1, quantity, time, time_1, **_):
#     return {
#         "identify": quantity,
#         "filter": [
#             {"attr": category, "op": "=", "value": category_1},
#             {"attr": time, "op": "=", "value": time_1},
#         ],
#     }


# @template("Which {category} has the highest {quantity} in {time_1}?")
# def ctq_max(category, quantity, time_1, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": "time", "op": "=", "value": time_1},
#             {
#                 "attr": quantity,
#                 "op": "=",
#                 "value": {"operation": "max", "attribute": quantity},
#             },
#         ],
#     }


# @template("Which {category} has the lowest {quantity} in {time_1}?")
# def ctq_min(category, quantity, time_1, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": "time", "op": "=", "value": time_1},
#             {
#                 "attr": quantity,
#                 "op": "=",
#                 "value": {"operation": "min", "attribute": quantity},
#             },
#         ],
#     }


# @template(
#     "What is the difference of {quantity} between {category_1} and {category_2} in {time_1}?"
# )
# def ctq_diff(category, category_1, category_2, quantity, time_1, **_):
#     return {
#         "compare": quantity,
#         "list": [
#             {
#                 "identify": category,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_1},
#                     {"attr": "time", "op": "=", "value": time_1},
#                 ],
#             },
#             {
#                 "identify": category,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_2},
#                     {"attr": "time", "op": "=", "value": time_1},
#                 ],
#             },
#         ],
#     }


# @template("What is the trend of the {quantity} of {category_1}?")
# def ctq_trend(category, category_1, quantity, **_):
#     return {
#         "trend": quantity,
#         "filter": [{"attr": category, "op": "=", "value": category_1}],
#     }


# @template("What is the sum of {quantity} of {category_1} and {category_2} in {time_1}?")
# def ctq_sum(category, category_1, category_2, quantity, time_1, **_):
#     return {
#         "sum": quantity,
#         "list": [
#             {
#                 "identify": quantity,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_1},
#                     {"attr": "time", "op": "=", "value": time_1},
#                 ],
#             },
#             {
#                 "identify": quantity,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_2},
#                     {"attr": "time", "op": "=", "value": time_1},
#                 ],
#             },
#         ],
#     }


# @template("Which {category} has the highest {quantity} from {time_1} to {time_2}?")
# def ctq_max_range(category, quantity, time_1, time_2, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#             {
#                 "attr": quantity,
#                 "op": "=",
#                 "value": {"operation": "max", "attribute": quantity},
#             },
#         ],
#     }


# @template("Which {category} has the lowest {quantity} from {time_1} to {time_2}?")
# def ctq_min_range(category, quantity, time_1, time_2, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#             {
#                 "attr": quantity,
#                 "op": "=",
#                 "value": {"operation": "min", "attribute": quantity},
#             },
#         ],
#     }


# @template(
#     "What is the sum of {quantity} of {category_1} and {category_2} from {time_1} to {time_2}?"
# )
# def ctq_sum_range(category, category_1, category_2, quantity, time_1, time_2, **_):
#     return {
#         "sum": quantity,
#         "list": [
#             {
#                 "identify": quantity,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_1},
#                     {"attr": "time", "op": ">=", "value": time_1},
#                     {"attr": "time", "op": "<=", "value": time_2},
#                 ],
#             },
#             {
#                 "identify": quantity,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_2},
#                     {"attr": "time", "op": ">=", "value": time_1},
#                     {"attr": "time", "op": "<=", "value": time_2},
#                 ],
#             },
#         ],
#     }


# @template(
#     "What is the difference of {quantity} between {category_1} and {category_2} from {time_1} to {time_2}?"
# )
# def ctq_diff_range(category, category_1, category_2, quantity, time_1, time_2, **_):
#     return {
#         "compare": quantity,
#         "list": [
#             {
#                 "identify": category,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_1},
#                     {"attr": "time", "op": ">=", "value": time_1},
#                     {"attr": "time", "op": "<=", "value": time_2},
#                 ],
#             },
#             {
#                 "identify": category,
#                 "filter": [
#                     {"attr": category, "op": "=", "value": category_2},
#                     {"attr": "time", "op": ">=", "value": time_1},
#                     {"attr": "time", "op": "<=", "value": time_2},
#                 ],
#             },
#         ],
#     }


# @template(
#     "What is the trend of the {quantity} of {category_1} from {time_1} to {time_2}?"
# )
# def ctq_trend_range(category, category_1, quantity, time_1, time_2, **_):
#     return {
#         "trend": quantity,
#         "filter": [
#             {"attr": category, "op": "=", "value": category_1},
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#         ],
#     }


# added


# @template(
#     "What is the trend of the {quantity} of {category_1} and {category_2} from {time_1} to {time_2}?"
# )
# def ctq_multi_trend_range(category, category_1, category_2, quantity, time_1, time_2, **_):
#     return {
#         "trend": quantity,
#         "filter": [
#             {"attr": category, "op": "in", "value": [category_1, category_2]},
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#         ],
#     }


# @template(
#     "Among {category_1} and {category_2}, what is the {category} with the highest {quantity} from {time_1} to {time_2}?"
# )
# def ctq_max_value_choice_range(category, category_1, category_2, quantity, time_1, time_2, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": category, "op": "in", "value": [category_1, category_2]},
#             {"attr": quantity, "op": "=", "value": {"operation": "max", "attribute": quantity}},
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#         ],
#     }

# @template(
#     "Among {category_1}, {category_2}, and {category_3}, what is the {category} with the highest {quantity} from {time_1} to {time_2}?"
# )
# def ctq_max_value_3_choice_range(category, category_1, category_2, category_3, quantity, time_1, time_2, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": category, "op": "in", "value": [category_1, category_2, category_3]},
#             {"attr": quantity, "op": "=", "value": {"operation": "max", "attribute": quantity}},
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#         ],
#     }

# @template(
#     "Among {category_1} and {category_2}, what is the {category} with higher {quantity} from {time_1} to {time_2}?"
# )
# def ctq_top_value_2_choice_range(category, category_1, category_2, rank_1, quantity, time_1, time_2, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": category, "op": "in", "value": [category_1, category_2]},
#             {"attr": {'operation': 'rank', 'attribute': quantity}, "op": "in", "value": [1, rank_1]},
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#         ],
#     }

# @template(
#     "What is the {category} with the top-{rank_1} {quantity} from {time_1} to {time_2}?"
# )
# def ctq_top_3_choice_range(category, rank_1, quantity, time_1, time_2, **_):

#     if rank_1 == 1:
#         op = '='
#         value = 1

#     return {
#         "identify": category,
#         "filter": [
#             {"attr": {'operation': 'rank', 'attribute': quantity}, "op": "in", "value": [1, rank_1]},
#             {"attr": "time", "op": ">=", "value": time_1},
#             {"attr": "time", "op": "<=", "value": time_2},
#         ],
#     }


# @template(
#     "What is the {category} with the top-{rank_1} {quantity} in {time_1}?"
# )
# def ctq_top_3_choice_single_time(category, rank_1, quantity, time_1, **_):
#     return {
#         "identify": category,
#         "filter": [
#             {"attr": {'operation': 'rank', 'attribute': quantity}, "op": "in", "value": [1, rank_1]},
#             {"attr": "time", "op": "=", "value": time_1}
#         ],
#     }

@template(
    "What is the average {quantity}?"
)
def avg_quan(quantity, **_):
    return {
        "identify": {
            "operation": "avg",
            "attribute": quantity
        }
    }

@template(
    "What is the average {quantity} from {time_1} to {time_2}?"
)
def avg_quan_time_range(quantity, time_1, time_2, **_):
    return {
        "identify": {
            "operation": "avg",
            "attribute": quantity
        },
        "filter": [
            {"attr": "time", "op": ">=", "value": time_1},
            {"attr": "time", "op": "<=", "value": time_2}
        ]
    }

@template(
    "What is the average {quantity} in {time_1}?"
)
def avg_quan_time_single_range(quantity, time_1, **_):
    return {
        "identify": {
            "aggregate": "avg",
            "attribute": quantity
        },
        "filter": [
            {"attr": "time", "op": "=", "value": time_1}
        ]
    }


In [29]:
import random
from typing import Callable, Tuple


def construct_question(attributes, template: Tuple[str, Callable]):
    c, t, q = attributes
    sentence_t, query_t = template

    params = {
        "category": c["name"],
        "quantity": q["name"],
        "time_1": t["span"][0],
        "time_2": t["span"][1],
    }
    params["category_1"], params["category_2"], params['category_3'] = random.sample(c["choice"], k=3)
    params['rank_1'] = random.randint(1, 3)

    sentence = sentence_t.format(**params).replace("_", " ")
    query = query_t(**params)

    question = f"""
    请用另一种说法表达以下问题：
    {sentence}
    """.strip()

    sentence_refined = ''.join(
        chat_complete(
            system="You are a helpful assistant that has a good command of language.",
            question=question,
            proxy="http://127.0.0.1:7890",
        )
    )

    return sentence, sentence_refined, query


In [30]:
import json
import random
import tqdm
import pandas as pd

df = pd.read_csv("ctq_attribute.csv")
res = pd.DataFrame(columns=["input", "output"])

file_output = 'ctq_with_avg.csv'

N = 5000
N = 50

with tqdm.tqdm(total=N) as pbar:
    success = 0
    error = 0
    pbar.set_description(f"Success: {success}, Error: {error}")
    for _ in range(N):
        _, c, t, q = df.sample().iloc[0]
        try:
            c = eval(c)
            t = eval(t)
            q = eval(q)
            template = random.choice(templates)

            print(c,t,q, template)

            sentence, sentence_refined, query = construct_question((c, t, q), template)

            input_ = {
                "attributes": [c, t, q],
                "sentence": sentence_refined,
            }
            output = query

            print(output)

            res = pd.concat(
                [
                    res,
                    pd.DataFrame(
                        {"input": [json.dumps(input_)], "output": [json.dumps(output)]}
                    ),
                ],
                ignore_index=True,
            )
            res.to_csv(file_output, index=False)
            success += 1
            pbar.update(1)
        except Exception:
            error += 1

        pbar.set_description(f"Success: {success}, Error: {error}")


Success: 0, Error: 0:   0%|          | 0/50 [00:00<?, ?it/s]

{'type': 'categorical', 'name': 'crime_type', 'choice': ['Assault', 'Burglary', 'Robbery', 'Larceny-Theft', 'Rape', 'Homicide', 'Drug Offenses', 'Fraud', 'Vandalism', 'Motor Vehicle Theft']} {'type': 'temporal', 'span': ['1990', '2021']} {'type': 'quantitative', 'name': 'number_of_crimes_reported', 'range': ['0', '100000']} ('What is the average {quantity} from {time_1} to {time_2}?', <function avg_quan_time_range at 0x147251cf0>)


Success: 1, Error: 0:   2%|▏         | 1/50 [00:03<02:49,  3.46s/it]

{'identify': {'operation': 'avg', 'attribute': 'number_of_crimes_reported'}, 'filter': [{'attr': 'time', 'op': '>=', 'value': '1990'}, {'attr': 'time', 'op': '<=', 'value': '2021'}]}
{'type': 'categorical', 'name': 'brand', 'choice': ['Chanel', 'Louis Vuitton', 'Gucci', 'Prada', 'Dior', 'Yves Saint Laurent', 'Hermes', 'Burberry', 'Versace']} {'type': 'temporal', 'span': ['2010', '2021']} {'type': 'quantitative', 'name': 'price', 'range': ['100', '5000']} ('What is the average {quantity}?', <function avg_quan at 0x1472524d0>)


Success: 2, Error: 0:   4%|▍         | 2/50 [00:05<02:05,  2.61s/it]

...

{'identify': {'operation': 'avg', 'attribute': 'square footage'}}
{'type': 'categorical', 'name': 'product_category', 'choice': ['Electronics', 'Apparel', 'Books', 'Beauty', 'Sports', 'Home & Furniture', 'Toys', 'Food & Beverage']} {'type': 'temporal', 'span': ['2020-01-01', '2021-12-31']} {'type': 'quantitative', 'name': 'sales', 'range': ['100', '10000']} ('What is the average {quantity} in {time_1}?', <function avg_quan_time_single_range at 0x1472f9360>)


Success: 48, Error: 1:  96%|█████████▌| 48/50 [02:51<00:05,  2.73s/it]

{'identify': {'aggregate': 'avg', 'attribute': 'sales'}, 'filter': [{'attr': 'time', 'op': '=', 'value': '2020-01-01'}]}
{'type': 'categorical', 'name': 'media_type', 'choice': ['TV Show', 'Movie', 'Documentary', 'Anime', 'Reality TV', 'Talk Show']} {'type': 'temporal', 'span': ['2015', '2022']} {'type': 'quantitative', 'name': 'IMDb_rating', 'range': ['0', '10']} ('What is the average {quantity} in {time_1}?', <function avg_quan_time_single_range at 0x1472f9360>)


Success: 49, Error: 1:  98%|█████████▊| 49/50 [02:54<00:03,  3.56s/it]

{'identify': {'aggregate': 'avg', 'attribute': 'IMDb_rating'}, 'filter': [{'attr': 'time', 'op': '=', 'value': '2015'}]}


In [19]:
def strip(s):    
    o = json.loads(s)    
    o["sentence"] = o["sentence"].strip().split("\n")[-1]
    return json.dumps(o)

df = pd.read_csv(file_output)
df.input = df.input.apply(strip)
df.to_csv(file_output, index=False)